<!-- TABS -->
# Connecting listeners

In [ ]:
# <testing: >
from superduperdb import superduper, Document
from superduperdb.backends.mongodb import Collection
from superduperdb.ext.numpy import array
import numpy as np

db = superduper('mongomock://temp')
select = Collection("data").find()

image_array = array("float64", shape=(256, 256, 3))
db.add(image_array)

datas = []
for i in range(10):
    data = {
        "x": f"data_{i}",
    }
    datas.append(data)

db.execute(Collection("data").insert_many([Document(data) for data in datas]))

In [ ]:
# <testing: >
from superduperdb import ObjectModel
model1 = ObjectModel(identifier="model1", object=lambda x: "model1_" + str(x))
model2 = ObjectModel(identifier="model2", object=lambda x1, x2: {"x1": x1, "x2":x2})

In [ ]:
# <tab: One By One>
listener1 = model1.to_listener(key="x", select=select, identifier="l1")
db.add(listener1)
listener2 = model2.to_listener(key=["x", listener1.outputs], select=select, identifier="l2")
db.add(listener2)

In [ ]:
# <tab: Graph>
from superduperdb.components.graph import Graph, document_node, input_node
in_ = document_node('x')
output1 = model1(x=in_['x'], outputs="l1")
output2 = model2(x1=output1, x2=in_["x"], outputs="l2")
listener_stack = output2.to_listeners(select=select, identifier="listener")
db.add(listener_stack)